In [11]:
import matplotlib
import cv2
import PyQt5
import numpy as np
import matplotlib as plt
import math
from cfg import *


In [12]:
def normalize(x, m, seed):
    return 1 / (np.sqrt(np.pi * 2) * seed) * np.e ** (-np.power((x - m) / seed, 2) / 2)


def Gaus_filter(size, sigma=1):
    array_1d = np.empty(size)
    start = size // 2
    # initialized(好像可用linspace取代)
    for index in range(size):
        array_1d[index] = index - start
    print(array_1d)
    # normalized
    for index in range(size):
        array_1d[index] = normalize(array_1d[index], 0, sigma)
    array_2d = np.outer(array_1d.T, array_1d.T)
    array_2d *= 1.0 / array_2d.max()

    print(array_2d)
    return array_2d  # return filter

# make sure convert to grayscale before calling convolution


def convolution(image, filter, avg=False):

    img_rows, img_cols = image.shape
    fil_row, fil_col = filter.shape
    result = np.zeros((img_rows, img_cols))  # for output

    # padding block
    padding_r = int((fil_row - 1) / 2)
    padding_c = int((fil_col - 1) / 2)
    pad_img = np.zeros(((padding_r*2) + img_rows, (padding_c*2) + img_cols))
    pad_img[padding_r:pad_img.shape[0] - padding_r,
            padding_c:pad_img.shape[1] - padding_c] = image
    # conv operation
    for row in range(img_rows):
        for col in range(img_cols):
            # filter 走到之處九宮格相乘再全sum起來
            result[row, col] = np.sum(
                filter * pad_img[row: row + fil_row, col: col + fil_col])
            if avg:
                # divide total pixels number
                result[row, col] /= filter.shape[0] * filter.shape[1]
    return result


def gaussian_blur(img, filter_size):
    sigma1 = math.sqrt(filter_size)  # square root the filter_size
    filter = Gaus_filter(filter_size,  math.sqrt(filter_size))
    return convolution(img, filter, avg=True)

In [13]:



def sobel_edge_detection(image, filter, verbose=False):
    new_image_x = convolution(image, filter, verbose)

    if verbose:
        plt.imshow(new_image_x, cmap='gray')
        plt.title("Horizontal Edge")
        plt.show()

    new_image_y = convolution(image, np.flip(filter.T, axis=0), verbose)

    if verbose:
        plt.imshow(new_image_y, cmap='gray')
        plt.title("Vertical Edge")
        plt.show()

    '''
    gradient_magnitude = np.sqrt(
        np.square(new_image_x) + np.square(new_image_y))

    gradient_magnitude *= 255.0 / gradient_magnitude.max()
    '''
    new_image_x *= 255 / new_image_x.max()
    if verbose:
        plt.imshow(gradient_magnitude, cmap='gray')
        plt.title("Gradient Magnitude")
        plt.show()

    #return gradient_magnitude
    return new_image_x


In [14]:
gau2 = cv2.imread(q3_folder + 'House.jpg')
gau2 = cv2.cvtColor(gau2, cv2.COLOR_BGR2GRAY)
filter = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
image = gaussian_blur(gau2, 3)
image = sobel_edge_detection(image, filter)
'''
cv2.imshow = ('test', image)
cv2.waitKey(0)
'''
cv2.imwrite('sobel.jpg' , image)


[-1.  0.  1.]
[[0.71653131 0.84648172 0.71653131]
 [0.84648172 1.         0.84648172]
 [0.71653131 0.84648172 0.71653131]]


True